# Задание 2. Итерационные методы решения линейных систем

In [3]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List, Tuple, Union
import warnings
from funcs import Matrix, Gauss
warnings.filterwarnings('ignore')

In [8]:
def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))


def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [(latexify(arg) if isinstance(arg, np.ndarray) else str(arg)) for arg in argv] + ['$$'])
    display(Markdown(res))


def get_latex_column(arg: str, n: int) -> str:
    """
    Prepares a string with LaTex syntax for a column of `arg` with indeces form 1 to `n`
    """
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def print_system_output(A: Matrix, s: str, B: Matrix, X: Matrix, n: int) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = np.matmul(A, X)
    print_tex('X^* = ', get_latex_column("x^*", n), '=', X.tex())
    print_tex(rf'{s} \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n), r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))

In [9]:
n = 4
A = Matrix(np.random.rand(n, n) * 10)
B = Matrix(np.random.rand(n, n) * 10)

In [10]:
print_tex(A, r'\times', B, '=', A @ B)

$$ \begin{pmatrix} 9.858803 & 9.958730 & 5.834920 & 0.581073 \\ 0.274446 & 2.253814 & 3.204976 & 5.073564 \\ 8.110552 & 5.416011 & 1.805083 & 9.767494 \\ 1.500910 & 1.169857 & 8.365850 & 4.896689 \end{pmatrix} \times \begin{pmatrix} 4.382954 & 1.044127 & 2.230877 & 9.346110 \\ 7.330379 & 9.078044 & 1.228830 & 7.939186 \\ 6.014221 & 5.401522 & 6.805697 & 6.075581 \\ 7.706538 & 8.028049 & 3.136365 & 5.185765 \end{pmatrix} = \begin{pmatrix} 155.782500 & 136.881959 & 75.764518 & 209.669499 \\ 76.099245 & 78.789352 & 41.106453 & 66.240858 \\ 161.379321 & 145.799346 & 67.668272 & 180.419684 \\ 103.204509 & 96.686345 & 77.079151 & 99.535873 \end{pmatrix} $$